#### 1. Import pandas library

In [1]:
import pandas as pd
import numpy as np

#### 2. Import BigQuery library


In [2]:
from google.cloud import bigquery

#### 3. Create a connection to BigQuery

In [3]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/Sosa/Downloads/ironhack_service_account_big_query.json"
client = bigquery.Client()

#### 4. Import data from the users table from the stackoverflow public database for United Kingdom only

In [4]:
sql = '''
SELECT 
    *
FROM 
    `bigquery-public-data.stackoverflow.users`
WHERE
    location LIKE '%United Kingdom%'

'''

job_query = client.query(query= sql)

df = job_query.to_dataframe()

In [5]:
df.shape

(66832, 13)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66832 entries, 0 to 66831
Data columns (total 13 columns):
id                   66832 non-null int64
display_name         66832 non-null object
about_me             24957 non-null object
age                  0 non-null object
creation_date        66832 non-null datetime64[ns, UTC]
last_access_date     66832 non-null datetime64[ns, UTC]
location             66832 non-null object
reputation           66832 non-null int64
up_votes             66832 non-null int64
down_votes           66832 non-null int64
views                66832 non-null int64
profile_image_url    44707 non-null object
website_url          20877 non-null object
dtypes: datetime64[ns, UTC](2), int64(5), object(6)
memory usage: 6.6+ MB


#### 5. Rename id column to user_id

In [7]:
df = df.rename(columns = {'id': 'user_id'})

df.columns

Index(['user_id', 'display_name', 'about_me', 'age', 'creation_date',
       'last_access_date', 'location', 'reputation', 'up_votes', 'down_votes',
       'views', 'profile_image_url', 'website_url'],
      dtype='object')

#### 6. Import from the stackoverflow_posts table all items which have up to 20 view counts and only select 'id', 'owner_user_id', 'score', 'view_count', 'comment_count', 'favorite_count'

In [8]:
qry= '''
SELECT
    id,
    owner_user_id,
    score,
    view_count,
    comment_count,
    favorite_count,

FROM
    `bigquery-public-data.stackoverflow.stackoverflow_posts` 

WHERE
    view_count <= 20
'''

query_job = client.query(query= qry)

In [9]:
df2 = query_job.to_dataframe()

In [10]:
df2.head()

,id,owner_user_id,score,view_count,comment_count,favorite_count
0,23154134,NaN,1,20,0,NaN
1,37516574,NaN,0,7,0,NaN
2,37319831,NaN,1,18,0,NaN
3,31140287,NaN,1,16,3,NaN
4,36546336,NaN,0,8,0,NaN


#### 7. Rename id column to post_id and owner_user_id to user_id

In [11]:
df2 = df2.rename(columns= {'id': 'post_id', 'owner_user_id': 'user_id'})

df2.head()

,post_id,user_id,score,view_count,comment_count,favorite_count
0,23154134,NaN,1,20,0,NaN
1,37516574,NaN,0,7,0,NaN
2,37319831,NaN,1,18,0,NaN
3,31140287,NaN,1,16,3,NaN
4,36546336,NaN,0,8,0,NaN


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [62]:
user_columns = df[['user_id', 'reputation', 'views', 'up_votes','down_votes']]
posts_columns = df2[['post_id', 'score', 'user_id', 'view_count', 'comment_count']]

#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [70]:
user_posts = pd.merge(user_columns, posts_columns).astype(int)

user_posts.head()

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,115237,5580,381,641,15,23933814,0,20,1
1,131809,30144,1923,1264,2098,37342151,0,16,8
2,253056,183790,23287,14348,17059,36934736,2,20,0
3,1165020,1018,256,680,10,34854096,0,20,0
4,1499072,2073,340,1964,13,34118493,0,13,0


#### 9. How many missing values do you have in your merged dataframe? 

In [71]:
user_posts.isna().sum()

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [72]:
user_posts = user_posts.drop('post_id', axis=1)

In [73]:
stats = user_posts.describe().transpose()
stats['IQR'] = stats['75%'] - stats['25%']
stats
#user_posts.head()

,count,mean,std,min,25%,50%,75%,max,IQR
user_id,4702.0,2.290522e+06,1.903187e+06,199.0,630530.0,1723133.0,3632206.00,6452203.0,3001676.00
reputation,4702.0,5.457066e+03,1.678706e+04,1.0,231.0,1140.5,4311.00,279695.0,4080.00
views,4702.0,4.960581e+02,1.258679e+03,0.0,38.0,138.0,419.00,27257.0,381.00
up_votes,4702.0,3.691672e+02,7.725259e+02,0.0,15.0,90.0,354.75,14348.0,339.75
down_votes,4702.0,4.030689e+01,3.215170e+02,0.0,0.0,3.0,16.00,17059.0,16.00
score,4702.0,1.156954e-01,5.435827e-01,-7.0,0.0,0.0,0.00,4.0,0.00
view_count,4702.0,1.538835e+01,3.797492e+00,2.0,13.0,16.0,19.00,20.0,6.00
comment_count,4702.0,8.966397e-01,1.593820e+00,0.0,0.0,0.0,1.00,15.0,1.00


In [74]:



low_variance = []

for col in user_posts._get_numeric_data():
    minimum = min(user_posts[col])
    ninety_per = np.percentile(user_posts[col], 90)
    if ninety_per == minimum:
        low_variance.append(col)
user_posts = user_posts.drop(low_variance, axis=1)


In [76]:
outliers = pd.DataFrame(columns=user_posts.columns)
for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = user_posts[(user_posts[col] < lower) |
                   (user_posts[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)
    
outliers.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,Outlier,comment_count,down_votes,reputation,score,up_votes,user_id,view_count,views
1,reputation,8,2098,30144,0,1264,131809,16,1923
2,reputation,0,17059,183790,2,14348,253056,20,23287
8,reputation,1,623,18981,0,787,1822164,20,2806
24,reputation,0,28,27410,0,727,911930,16,2700
25,reputation,2,28,27410,0,727,911930,20,2700


In [77]:
outliers.to_csv('outliers.csv')